# Interaction between characters

In [1]:
import sys
sys.path.insert(1, '../src')

from book_entities import *
from book_entities_embeddings_getters import *

## Book entities

In [2]:
# cleaned entities
textually_close_merged_book_cleaned_entities, lax_merged_book_cleaned_entities = get_book_entities(
    f'../data/book_entities/grouped_token_cleaned.csv')

In [4]:
for i in textually_close_merged_book_cleaned_entities:
    print(i)

<BookListEntity : [
	honorific: '['comte']'
	first: '['altamira']' 
	middle: '[]' 
	last: '['altamira']' 
	suffix: '[]'
	nickname: '[]'
	gender: 'None'
	all adresses: '['Altamira', 'comte Altamira']'
]>
<BookListEntity : [
	honorific: '[]'
	first: '['amanda']' 
	middle: '[]' 
	last: '[]' 
	suffix: '[]'
	nickname: '[]'
	gender: 'F'
	all adresses: '['Amanda']'
]>
<BookListEntity : [
	honorific: '['m']'
	first: '['']' 
	middle: '[]' 
	last: '['appert']' 
	suffix: '[]'
	nickname: '[]'
	gender: 'M'
	all adresses: '['M Appert']'
]>
<BookListEntity : [
	honorific: '[]'
	first: '['bonaparte']' 
	middle: '[]' 
	last: '[]' 
	suffix: '[]'
	nickname: '[]'
	gender: 'None'
	all adresses: '['Bonaparte']'
]>
<BookListEntity : [
	honorific: '['abbé', 'm']'
	first: '['']' 
	middle: '[]' 
	last: '['castanède']' 
	suffix: '[]'
	nickname: '[]'
	gender: 'M'
	all adresses: '['abbé Castanède', 'M Castanède']'
]>
<BookListEntity : [
	honorific: '[]'
	first: '['caylus']' 
	middle: '[]' 
	last: '[]' 
	suffix: '[

## Paragraphs

In [41]:
from bs4 import BeautifulSoup

In [2]:
# Reading the data inside the xml
# file to a variable under the name
# data
with open('../data/book/stendhal_rougenoir_xml.xml', 'r') as f:
	data = f.read()

In [3]:
# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object
Bs_data = BeautifulSoup(data, "xml")

# Finding all instances of tag
# `unique`
b_p = Bs_data.find_all('p')
b_rend = Bs_data.find_all(rend=['right', 'right i'])

paragraphs = [i for i in b_p if i not in b_rend]
paragraphs = [p.contents[0] for p in paragraphs]

In [4]:
paragraphs[:3][1][0]

'V'

### dialogue et narration

In [36]:
dialogue =[]
narration = []
temp_dialogue = []
for p in paragraphs :
    if isinstance(p, str) :
        if p[0] == '–' :
            temp_dialogue.append(p)
        else :
            if len(temp_dialogue)>0 :
                dialogue.append(temp_dialogue)
                temp_dialogue=[]
            narration.append(p)
        

In [39]:
len(dialogue)

570

In [40]:
len(narration)

2495